In [51]:
#!pip install langchain
#!pip install sentence-transformers
#!pip install faiss-cpu
#!pip install bs4
#!pip install replicate
#!pip install langchain_community
#!pip install -U langchain-huggingface

In [52]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
#from langchain_huggingface import HuggingFaceEmbeddings
import bs4



In [53]:
import requests
import json

url = "http://localhost:11434/api/chat"
def llama3(prompt):
    data = {
        "model": "llama3",
        "messages": [
            {
                "role": "user",
                "content": prompt

            }
        ],
        "stream": False,
    }

    headers = {
        "Content-Type": "application/json"
    }

    response = requests.post(url, headers=headers, json=data)
    return response.json()["message"]["content"]




In [54]:
from langchain_community.chat_models import ChatOllama
llm = ChatOllama(model="llama3", temperature = 0)

from langchain.chains import ConversationalRetrievalChain

chat_chain = ConversationalRetrievalChain.from_llm(llm, 
                                                   vectorstore.as_retriever(), 
                                                   return_source_documents=True)

In [55]:
chat_history = []
question = "What's new with Llama 3?"
result = chat_chain({"question": question, "chat_history": chat_history})
print(result['answer'])

According to the provided context, the main changes and features introduced by Meta's Llama 3 are:

1. Four new open LLM models based on the Llama 2 architecture.
2. Two sizes: 8B and 70B parameters, each with base (pre-trained) and instruct-tuned versions.
3. All variants can be run on various types of consumer hardware.
4. Context length of 8K tokens.

Additionally, a new tokenizer is introduced that expands the vocabulary size to 128,256 (from 32K tokens in Llama 2), which allows for more efficient encoding of text and potentially stronger multilingualism.


In [41]:
#Article from NTSB about crashes
loader = WebBaseLoader(["https://www.flyingmag.com/news/ntsb-releases-details-on-2-lockheed-12a-crashes/"])
#Test file - What's new about Llama 3
loader2 = WebBaseLoader(["https://huggingface.co/blog/llama3"])
docs = loader.load()
docs2 = loader2.load()
# Split the document into chunks with a specified chunk size
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
#Loading multiple documents
all_splits = text_splitter.split_documents(docs + docs2)


# Store the document into a vector store with a specific embedding model
vectorstore = FAISS.from_documents(all_splits, HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2"))

In [50]:
#RAG from NTSB article
chat_chain = ConversationalRetrievalChain.from_llm(llm, 
                                                   vectorstore.as_retriever(), 
                                                   return_source_documents=True)
chat_history = []
question = "Can you tell me about the Chino, California incident?"
result = chat_chain({"question": question, "chat_history": chat_history})
print(result['answer'])

According to the context, on June 15 in Chino, California, a Lockheed 12A aircraft (N93R) crashed during takeoff while participating in a Father's Day airshow. The pilot and copilot were killed in the accident. The NTSB investigation found that the tailwheel lock control lever was in the locked position, but the locking tab on the control-wheel assembly appeared to be unlocked. Additionally, there was no fluid leakage observed.


In [47]:
#RAG from test document
chat_history = []
question = "What's new with Llama 3?"
result = chat_chain({"question": question, "chat_history": chat_history})
print(result['answer'])

According to the provided context, the main changes and features introduced by Meta's Llama 3 are:

1. Four new open LLM models based on the Llama 2 architecture.
2. Two sizes: 8B and 70B parameters, each with base (pre-trained) and instruct-tuned versions.
3. All variants can be run on various types of consumer hardware.
4. Context length of 8K tokens.

Additionally, a new tokenizer is introduced that expands the vocabulary size to 128,256 (from 32K tokens in Llama 2), which allows for more efficient encoding of text and potentially stronger multilingualism.
